# Classification for Machine Learning




In [1]:
import sys
sys.path.insert(1, '../../crystal-code-tools/crystal_functions/crystal_functions/')

from file_readwrite import Crystal_output, write_cry_properties, Crystal_input, write_cry_gui,write_cry_input, Density
from execute import runcry
from convert import cry_gui2pmg

from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.cif import CifWriter
from ase.visualize import view

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import SlabGenerator
import numpy as np
import pandas as pd

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
new_atom = 'Ca'

### Read the structures from the output files and replace with new_atom
This cell generates the structures list. This contains all the 4023 structures as pymatgen structure objects

In [67]:
cry_output = Crystal_output('data/classification/ml/mgo_confrand.out')
cry_output.config_analysis()


original_structure = cry_gui2pmg('data/classification/ml/mgo_confrand.gui')
structures = []
mg_atoms = []
ca_atoms = []
for j,substitutions in enumerate(cry_output.atom_type2):
    new_structure = original_structure.copy()
    for i in substitutions:
        new_structure.replace(i-1,new_atom)
    structures.append(new_structure)
    mg_atoms.append((np.array(cry_output.atom_type1[j])-1).tolist())
    ca_atoms.append((np.array(cry_output.atom_type2[j])-1).tolist())

### Shells

In [26]:
shells = np.unique(np.round(structures[0].distance_matrix[0],decimals=6),return_counts=True)[0].tolist()
display(shells)
np.unique(np.round(structures[0].distance_matrix[0],decimals=6),return_counts=True)[1]

[0.0,
 2.105,
 2.97692,
 3.645967,
 4.21,
 4.706923,
 5.156176,
 5.953839,
 6.315,
 6.656594,
 6.981495,
 7.291934,
 7.589685,
 7.876189,
 8.679137,
 8.930759,
 9.175482,
 9.873325,
 10.937901]

array([ 1,  6, 12,  8,  6, 24, 24, 12, 27, 12, 12,  8, 12, 24, 12,  3,  6,
        6,  1])

In [22]:
view(AseAtomsAdaptor().get_atoms(structures[0]))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Site surrounding

#### Second shell

# The magic happens here

Each element of the neighbours_spatial_dist_all (one element per structure) is a 2D array whose rows correspond to an atom and the array contains:
- 0 = atom number of the atom whose neighbours I'm analysing
- 1:6 = atomic numbers of the atoms in the first coord shell
- 7:19 = atomic numbers of the atoms in the second coord shell
- 20:28 = atomic numbers of the atoms in the third coord shell

These atomic numbers are ordered following a spatial analysis. They are written starting from the atoms below the atom 0 and moving upwards.

## Spatial distribution analysis

In [208]:
def cart2sph(x, y, z):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    if np.around(az,6) ==  np.around(2*np.pi,6) \
    or np.around(az,6) ==  -np.around(2*np.pi,6):
        az = 0.
    if np.around(az,6) < 0.:
        az = np.round(2*np.pi+az,6)
    return [round(az,6), round(el,6), round(r,6)]

In [300]:
max_shell = 2
centered_sph_coords = []
centered_sph_coords_structure = []
neighbours_spatial_dist = []
neighbours_spatial_dist_all = []

for k,structure in enumerate(structures[0:3]):
    neighbours_spatial_dist = []
    for j in range(structure.num_sites):
        centered_sph_coords = []
        for i in range(structure.num_sites):    
            translation_vector = structure.sites[j].distance_and_image(structure.sites[i])[1]
            new_cart_coords = structure.cart_coords[i]+(translation_vector*structure.lattice.abc)
            centered_cart_coords = new_cart_coords-structure.cart_coords[j] 
            centered_sph_coords.append(cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))        

        #centered_sph_coords_all.append(centered_sph_coords)
        centered_sph_coords = np.array(centered_sph_coords)


        atoms_shell = []
        for unique in shells:
            atoms_shell.append(np.where(centered_sph_coords[:,2] == unique)[0].tolist())

        neighbours_spatial_dist_atom = []
        for shell in range(max_shell+1):
            #a = np.array(centered_sph_coords)[atoms_shell[shell]]
            spatial_distribution = np.argsort(centered_sph_coords[atoms_shell[shell]][:,1]*10 + centered_sph_coords[atoms_shell[shell]][:,0])
            #print(np.array(structure.atomic_numbers)[np.array(atoms_shell[shell])[spatial_distribution]])
            neighbours_spatial_dist_atom.extend((np.array(structure.atomic_numbers)[np.array(atoms_shell[shell])[spatial_distribution]]).tolist())
            #THIS IS THE CONTROL LINE
            #print(centered_sph_coords[np.array(atoms_shell[shell])[spatial_distribution]])
        neighbours_spatial_dist.append(neighbours_spatial_dist_atom)
    neighbours_spatial_dist_all.append(neighbours_spatial_dist)
    #centered_sph_coords_structure = np.array(centered_sph_coords_all)


In [301]:
for i in np.unique(neighbours_spatial_dist,axis=0,return_counts=True)[0]:
    print(i)

[ 8 12 12 12 12 12 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 12 12 12 12 20  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 12 12 12 20 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 12 12 20 12 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 12 20 12 12 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 12 20 12 20 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 20 12 12 12 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 12 20 12 20 12 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 20 12 12 12 12 12  8  8  8  8  8  8  8  8  8  8  8  8]
[ 8 20 12 12 12 12 20  8  8  8  8  8  8  8  8  8  8  8  8]
[12  8  8  8  8  8  8 12 12 12 12 12 12 12 12 12 12 12 12]
[12  8  8  8  8  8  8 12 12 12 12 12 12 12 12 12 12 12 20]
[12  8  8  8  8  8  8 12 12 12 12 12 12 12 12 12 12 20 12]
[12  8  8  8  8  8  8 12 12 12 12 12 12 12 12 12 20 12 12]
[12  8  8  8  8  8  8 12 12 12 12 12 12 20 12 12 12 12 12]
[12  8  8  8  8  8  8 12 12 12 12 12 20 20 12 12 12 12 12]
[12  8  8  8  8  8  8 12 12 12 12 12 20 20 20 12 12 12 1

In [286]:
np.unique(neighbours_spatial_dist,axis=0,return_counts=True)[1]

array([18,  9,  9,  9,  9, 18,  9,  9, 18, 18,  9,  9,  9,  9, 18,  9,  9,
       18])

In [210]:
order = np.argsort(centered_sph_coords_all[0][:,2])
centered_sph_coords_all[0][order]

array([[ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  2.105   ],
       [ 4.712389,  0.      ,  2.105   ],
       [ 0.      , -1.570796,  2.105   ],
       [ 1.570796,  0.      ,  2.105   ],
       [ 0.      ,  1.570796,  2.105   ],
       [ 3.141593,  0.      ,  2.105   ],
       [ 0.      ,  0.785398,  2.97692 ],
       [ 3.926991,  0.      ,  2.97692 ],
       [ 4.712389, -0.785398,  2.97692 ],
       [ 4.712389,  0.785398,  2.97692 ],
       [ 2.356194,  0.      ,  2.97692 ],
       [ 3.141593, -0.785398,  2.97692 ],
       [ 1.570796, -0.785398,  2.97692 ],
       [ 0.785398,  0.      ,  2.97692 ],
       [ 1.570796,  0.785398,  2.97692 ],
       [ 0.      , -0.785398,  2.97692 ],
       [ 5.497787,  0.      ,  2.97692 ],
       [ 3.141593,  0.785398,  2.97692 ],
       [ 0.785398,  0.61548 ,  3.645967],
       [ 0.785398, -0.61548 ,  3.645967],
       [ 5.497787,  0.61548 ,  3.645967],
       [ 5.497787, -0.61548 ,  3.645967],
       [ 3.926991, -0.61548 ,  3.6

In [238]:
r_unique = np.unique(centered_sph_coords_all[0][:,2])
r_unique

array([ 0.      ,  2.105   ,  2.97692 ,  3.645967,  4.21    ,  4.706923,
        5.156176,  5.953839,  6.315   ,  6.656594,  6.981495,  7.291934,
        7.589685,  7.876189,  8.679137,  8.930759,  9.175482,  9.873325,
       10.937901])

In [239]:
shells

[0.0,
 2.105,
 2.97692,
 3.645967,
 4.21,
 4.706923,
 5.156176,
 5.953839,
 6.315,
 6.656594,
 6.981495,
 7.291934,
 7.589685,
 7.876189,
 8.679137,
 8.930759,
 9.175482,
 9.873325,
 10.937901]

In [212]:
atoms_shell = []
for unique in r_unique:
    atoms_shell.append(np.where(centered_sph_coords_all[0][:,2] == unique)[0].tolist())

In [240]:
atoms_shell = []
for unique in shells:
    atoms_shell.append(np.where(centered_sph_coords_all[0][:,2] == unique)[0].tolist())

In [230]:
a = centered_sph_coords_all[0][atoms_shell[2]]
a

array([[ 0.785398,  0.      ,  2.97692 ],
       [ 5.497787,  0.      ,  2.97692 ],
       [ 2.356194,  0.      ,  2.97692 ],
       [ 3.926991,  0.      ,  2.97692 ],
       [ 0.      ,  0.785398,  2.97692 ],
       [ 0.      , -0.785398,  2.97692 ],
       [ 3.141593,  0.785398,  2.97692 ],
       [ 3.141593, -0.785398,  2.97692 ],
       [ 1.570796,  0.785398,  2.97692 ],
       [ 1.570796, -0.785398,  2.97692 ],
       [ 4.712389,  0.785398,  2.97692 ],
       [ 4.712389, -0.785398,  2.97692 ]])

In [226]:
c = np.argsort(centered_sph_coords_all[0][atoms_shell[1]][:,1]*10 + centered_sph_coords_all[0][atoms_shell[1]][:,0])

In [231]:
c = np.argsort(centered_sph_coords_all[0][atoms_shell[2]][:,1]*10 + centered_sph_coords_all[0][atoms_shell[2]][:,0])

In [234]:
np.array(atoms_shell[2])[c]

array([56, 83, 74, 89, 27, 45, 51, 33, 54, 81, 72, 87])

In [233]:
np.array(structures[0].atomic_numbers)[np.array(atoms_shell[2])[c]]

array([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [161]:
b = np.argsort(a[:,1])
a[b]

array([[ 0.      , -1.570796,  2.105   ],
       [ 0.      ,  0.      ,  2.105   ],
       [ 3.141593,  0.      ,  2.105   ],
       [ 1.570796,  0.      ,  2.105   ],
       [-1.570796,  0.      ,  2.105   ],
       [ 0.      ,  1.570796,  2.105   ]])

In [146]:
np.array(structures[0].atomic_numbers)[order]

array([20,  8,  8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12,  8,  8,  8,  8,  8,  8,  8,  8, 20, 20, 20, 20, 20, 12,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 20, 12,
       20, 20, 20, 20, 12, 20, 20, 20,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8, 12, 12, 20, 12, 20, 20, 12, 20,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 12, 12, 12,  8,
        8,  8,  8,  8,  8, 12, 12, 12, 12, 12, 12,  8])

In [131]:
centered_sph_coords = []
for i in range(structures[0].num_sites):    
    structure_tmp = structures[0]

    translation_vector = structures[0].sites[0].distance_and_image(structures[0].sites[i])[1]

    
    new_cart_coords = structures[0].cart_coords[i]+(translation_vector*structures[0].lattice.abc)
    
    centered_cart_coords = new_cart_coords-structures[0].cart_coords[0] #This will be i
    
    centered_sph_coords.append(cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))
    print(structures[0].cart_coords[i],centered_cart_coords,cart2sph(centered_cart_coords[0],centered_cart_coords[1],centered_cart_coords[2]))

centered_sph_coords = np.array(centered_sph_coords)

[0. 0. 0.] [0. 0. 0.] [0.0, 0.0, 0.0]
[0.   0.   4.21] [0.   0.   4.21] [0.0, 1.570796, 4.21]
[ 0.    0.   -4.21] [ 0.    0.   -4.21] [0.0, -1.570796, 4.21]
[0.   4.21 0.  ] [0.   4.21 0.  ] [1.570796, 0.0, 4.21]
[0.   4.21 4.21] [0.   4.21 4.21] [1.570796, 0.785398, 5.953839]
[ 0.    4.21 -4.21] [ 0.    4.21 -4.21] [1.570796, -0.785398, 5.953839]
[ 0.   -4.21  0.  ] [ 0.   -4.21  0.  ] [-1.570796, 0.0, 4.21]
[ 0.   -4.21  4.21] [ 0.   -4.21  4.21] [-1.570796, 0.785398, 5.953839]
[ 0.   -4.21 -4.21] [ 0.   -4.21 -4.21] [-1.570796, -0.785398, 5.953839]
[4.21 0.   0.  ] [4.21 0.   0.  ] [0.0, 0.0, 4.21]
[4.21 0.   4.21] [4.21 0.   4.21] [0.0, 0.785398, 5.953839]
[ 4.21  0.   -4.21] [ 4.21  0.   -4.21] [0.0, -0.785398, 5.953839]
[4.21 4.21 0.  ] [4.21 4.21 0.  ] [0.785398, 0.0, 5.953839]
[4.21 4.21 4.21] [4.21 4.21 4.21] [0.785398, 0.61548, 7.291934]
[ 4.21  4.21 -4.21] [ 4.21  4.21 -4.21] [0.785398, -0.61548, 7.291934]
[ 4.21 -4.21  0.  ] [ 4.21 -4.21  0.  ] [-0.785398, 0.0, 5.953839]
[ 

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [112]:
centered_sph_coords[np.argsort(centered_sph_coords[:,2])]
np.array(structures[0].atomic_numbers)[np.argsort(centered_sph_coords[:,2])]

array([20, 20,  8,  8,  8, 12,  8, 12, 12, 12, 12, 12, 12, 12,  8, 12,  8,
       12,  8,  8,  8,  8,  8, 12,  8,  8, 12, 12, 12, 12, 12, 12, 12, 12,
        8,  8, 12,  8,  8,  8,  8,  8,  8,  8, 20, 20, 20, 20,  8, 20,  8,
        8,  8,  8,  8,  8,  8, 12, 20, 12, 12,  8,  8,  8, 12, 12, 12,  8,
       12,  8,  8,  8, 12, 12,  8,  8, 12,  8,  8, 12, 12, 12, 12, 12,  8,
        8,  8, 12, 20,  8,  8,  8,  8, 12,  8, 20, 20, 20, 20,  8,  8,  8,
       12,  8,  8,  8, 12, 12, 12,  8, 12, 12, 12, 12,  8, 12,  8,  8,  8,
        8,  8, 12,  8, 12,  8, 12, 12, 12, 12,  8, 12,  8,  8, 12, 12, 12,
       12, 12, 12, 12,  8,  8, 12,  8,  8,  8,  8,  8, 12,  8, 12,  8,  8,
       20, 20,  8,  8,  8, 12,  8,  8, 12, 12, 12,  8, 12,  8, 12, 12, 12,
       12,  8,  8,  8, 20, 20, 12,  8, 12,  8,  8,  8, 12, 12, 12,  8,  8,
        8,  8, 12, 12,  8,  8, 12,  8,  8, 12, 12,  8,  8, 12, 12, 20,  8,
       12,  8,  8, 12,  8, 12,  8,  8, 12, 12,  8, 12])

In [128]:
centered_sph_coords[:,0:2]

array([[ 0.      ,  0.      ],
       [ 0.      ,  1.570796],
       [ 0.      , -1.570796],
       [ 1.570796,  0.      ],
       [ 1.570796,  0.785398],
       [ 1.570796, -0.785398],
       [-1.570796,  0.      ],
       [-1.570796,  0.785398],
       [-1.570796, -0.785398],
       [ 0.      ,  0.      ],
       [ 0.      ,  0.785398],
       [ 0.      , -0.785398],
       [ 0.785398,  0.      ],
       [ 0.785398,  0.61548 ],
       [ 0.785398, -0.61548 ],
       [-0.785398,  0.      ],
       [-0.785398,  0.61548 ],
       [-0.785398, -0.61548 ],
       [ 3.141593,  0.      ],
       [ 3.141593,  0.785398],
       [ 3.141593, -0.785398],
       [ 2.356194,  0.      ],
       [ 2.356194,  0.61548 ],
       [ 2.356194, -0.61548 ],
       [-2.356194,  0.      ],
       [-2.356194,  0.61548 ],
       [-2.356194, -0.61548 ],
       [ 0.785398,  0.      ],
       [ 0.785398,  0.955317],
       [ 0.785398, -0.955317],
       [-1.249046,  0.      ],
       [-1.249046,  0.563943],
       [

In [129]:
np.unique(centered_sph_coords[:,0:2],axis=1,return_counts=True)

(array([[ 0.      ,  0.      ],
        [ 0.      ,  1.570796],
        [ 0.      , -1.570796],
        [ 1.570796,  0.      ],
        [ 1.570796,  0.785398],
        [ 1.570796, -0.785398],
        [-1.570796,  0.      ],
        [-1.570796,  0.785398],
        [-1.570796, -0.785398],
        [ 0.      ,  0.      ],
        [ 0.      ,  0.785398],
        [ 0.      , -0.785398],
        [ 0.785398,  0.      ],
        [ 0.785398,  0.61548 ],
        [ 0.785398, -0.61548 ],
        [-0.785398,  0.      ],
        [-0.785398,  0.61548 ],
        [-0.785398, -0.61548 ],
        [ 3.141593,  0.      ],
        [ 3.141593,  0.785398],
        [ 3.141593, -0.785398],
        [ 2.356194,  0.      ],
        [ 2.356194,  0.61548 ],
        [ 2.356194, -0.61548 ],
        [-2.356194,  0.      ],
        [-2.356194,  0.61548 ],
        [-2.356194, -0.61548 ],
        [ 0.785398,  0.      ],
        [ 0.785398,  0.955317],
        [ 0.785398, -0.955317],
        [-1.249046,  0.      ],
        

In [235]:

#structure = cry_gui2pmg('data/classification/LTS_2_B3LYP_Ahl_b.gui')


#structure = cry_gui2pmg('data/classification/mgo_scel.gui')
structure = structures[0]

#How many first/second neighbours can an atom have?
max_n_first_shell = 6
max_n_second_shell = 12
max_n_third_shell = 8

#This is the final one (list of lists)

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. Unique elements only (unique to all structures)
unique_neighbours = []

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. All elements
first_neigh_atoms_all = []
#list of lists. Each element has 0 central atom index, 1: surrounding atom index (-1 if there are not enough neighbours
#All elements
first_neigh_all = []

#list of lists. Each element has 0 central atom specie, 1: surrounding atom species (-1 if there are not enough neighbours
#for example: [8, 12, 12, 12, -1]. Unique elements only (unique to this structure)
first_neigh_atoms_unique = []
#list of lists. Each element has 0 central atom index, 1: surrounding atom index (-1 if there are not enough neighbours
# Unique elements only (unique to this structure)
first_neigh_unique = []

#structure = mgo
#structure = structures[126]


for i,atom in enumerate(structure.sites):
    first_neigh = []
    first_neigh_atoms = []
    
    second_neigh = []
    second_neigh_atoms = []
    
    third_neigh = []
    third_neigh_atoms = []
    for j,atom in enumerate(structure.sites):        

        
        #Check is the atom j is in the first shell in the unit cell
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[1]) < 0.1:
            first_neigh.append(j)
            first_neigh_atoms.append(structure.atomic_numbers[j])
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[2]) < 0.1:
            second_neigh.append(j)
            second_neigh_atoms.append(structure.atomic_numbers[j])
    
        if abs(structure.sites[i].distance(structure.sites[j]) - shells[3]) < 0.1:
            third_neigh.append(j)
            third_neigh_atoms.append(structure.atomic_numbers[j])
            
    #print(i,j,first_neigh_atoms)
    sort_list = np.argsort(np.array(first_neigh_atoms))
    sort_list_second = np.argsort(np.array(second_neigh_atoms))
    sort_list_third = np.argsort(np.array(third_neigh_atoms))
    
    #First neighbour index for the i-th atom sorted
    first_neigh = (np.array(first_neigh)[sort_list]).tolist()
    #First neighbour species for the i-th atom sorted
    first_neigh_atoms = (np.array(first_neigh_atoms)[sort_list]).tolist()
    
    #Second neighbour index for the i-th atom sorted
    second_neigh = (np.array(second_neigh)[sort_list_second]).tolist()
    #Second neighbour species for the i-th atom sorted
    second_neigh_atoms = (np.array(second_neigh_atoms)[sort_list_second]).tolist()
    
    #T neighbour index for the i-th atom sorted
    third_neigh = (np.array(third_neigh)[sort_list_third]).tolist()
    #T neighbour species for the i-th atom sorted
    third_neigh_atoms = (np.array(third_neigh_atoms)[sort_list_third]).tolist()

    #Prepend the atom index to the first neighbours list
    first_neigh.insert(0,i)
    #Prepend the atom specie to the first neighbours list
    first_neigh_atoms.insert(0,structure.atomic_numbers[i])
    
    #Add -1 to first_neigh and first_neigh_atoms to reach the same lenght for all vectors
    if len(first_neigh) < max_n_first_shell+1:
        n = max_n_first_shell+1 - len(first_neigh)
        first_neigh.extend([-1]*n)
        first_neigh_atoms.extend([-1]*n)
    
    #Same for the second shell
    if len(second_neigh) < max_n_second_shell:
        n = max_n_second_shell+1 - len(second_neigh)
        second_neigh.extend([-1]*n)
        second_neigh_atoms.extend([-1]*n)
        
    #Same for the third shell
    if len(third_neigh) < max_n_third_shell:
        n = max_n_third_shell+1 - len(third_neigh)
        third_neigh.extend([-1]*n)
        third_neigh_atoms.extend([-1]*n)
        
    #TMP: append second to first neigh
    first_neigh.extend(second_neigh)
    first_neigh_atoms.extend(second_neigh_atoms)
    first_neigh.extend(third_neigh)
    first_neigh_atoms.extend(third_neigh_atoms)
    
    
    
    first_neigh_atoms_all.append(first_neigh_atoms) 
    first_neigh_all.append(first_neigh)
    print(first_neigh)
first_neigh_atoms_unique.extend(np.unique(first_neigh_atoms_all,axis=0,return_index=True)[0].tolist())
index = np.unique(first_neigh_atoms_all,axis=0,return_index=True)[1]
first_neigh_unique.extend((np.array(first_neigh_all)[index]).tolist())
              

[0, 108, 110, 162, 180, 189, 195, 27, 33, 45, 51, 54, 56, 72, 74, 81, 83, 87, 89, 135, 137, 141, 143, 153, 155, 159, 161]
[1, 108, 109, 163, 181, 190, 196, 28, 34, 46, 52, 54, 55, 72, 73, 81, 82, 87, 88, 135, 136, 141, 142, 153, 154, 159, 160]
[2, 109, 110, 164, 182, 191, 197, 29, 35, 47, 53, 55, 56, 73, 74, 82, 83, 88, 89, 136, 137, 142, 143, 154, 155, 160, 161]
[3, 111, 113, 165, 183, 189, 192, 27, 30, 45, 48, 57, 59, 75, 77, 81, 83, 84, 86, 135, 137, 138, 140, 153, 155, 156, 158]
[4, 111, 112, 166, 184, 190, 193, 28, 31, 46, 49, 57, 58, 75, 76, 81, 82, 84, 85, 135, 136, 138, 139, 153, 154, 156, 157]
[5, 112, 113, 167, 185, 191, 194, 29, 32, 47, 50, 58, 59, 76, 77, 82, 83, 85, 86, 136, 137, 139, 140, 154, 155, 157, 158]
[6, 114, 116, 168, 186, 192, 195, 30, 33, 48, 51, 60, 62, 78, 80, 84, 86, 87, 89, 138, 140, 141, 143, 156, 158, 159, 161]
[7, 114, 115, 169, 187, 193, 196, 31, 34, 49, 52, 60, 61, 78, 79, 84, 85, 87, 88, 138, 139, 141, 142, 156, 157, 159, 160]
[8, 115, 116, 170, 188, 

[75, 111, 129, 153, 156, 183, 184, 21, 22, 45, 46, 48, 49, 81, 84, 99, 102, 3, 4, 189, 190, 192, 193, 207, 208, 210, 211]
[76, 112, 130, 154, 157, 184, 185, 22, 23, 46, 47, 49, 50, 82, 85, 100, 103, 4, 5, 190, 191, 193, 194, 208, 209, 211, 212]
[77, 113, 131, 155, 158, 183, 185, 21, 23, 45, 47, 48, 50, 83, 86, 101, 104, 3, 5, 189, 191, 192, 194, 207, 209, 210, 212]
[78, 114, 132, 156, 159, 186, 187, 24, 25, 48, 49, 51, 52, 84, 87, 102, 105, 6, 7, 192, 193, 195, 196, 210, 211, 213, 214]
[79, 115, 133, 157, 160, 187, 188, 25, 26, 49, 50, 52, 53, 85, 88, 103, 106, 7, 8, 193, 194, 196, 197, 211, 212, 214, 215]
[80, 116, 134, 158, 161, 186, 188, 24, 26, 48, 50, 51, 53, 86, 89, 104, 107, 6, 8, 192, 194, 195, 197, 210, 212, 213, 215]
[81, 108, 111, 135, 153, 189, 190, 27, 28, 45, 46, 54, 57, 72, 75, 0, 1, 3, 4, 162, 163, 165, 166, 180, 181, 183, 184]
[82, 109, 112, 136, 154, 190, 191, 28, 29, 46, 47, 55, 58, 73, 76, 1, 2, 4, 5, 163, 164, 166, 167, 181, 182, 184, 185]
[83, 110, 113, 137, 155, 

[146, 36, 38, 65, 68, 92, 101, 119, 122, 128, 131, 171, 173, 174, 176, 198, 200, 207, 209, 18, 20, 21, 23, 9, 11, 12, 14]
[147, 39, 40, 66, 69, 93, 102, 120, 123, 129, 132, 174, 175, 177, 178, 201, 202, 210, 211, 21, 22, 24, 25, 12, 13, 15, 16]
[148, 40, 41, 67, 70, 94, 103, 121, 124, 130, 133, 175, 176, 178, 179, 202, 203, 211, 212, 22, 23, 25, 26, 13, 14, 16, 17]
[149, 39, 41, 68, 71, 95, 104, 122, 125, 131, 134, 174, 176, 177, 179, 201, 203, 210, 212, 21, 23, 24, 26, 12, 14, 15, 17]
[150, 42, 43, 63, 69, 96, 105, 117, 123, 126, 132, 171, 172, 177, 178, 204, 205, 213, 214, 18, 19, 24, 25, 9, 10, 15, 16]
[151, 43, 44, 64, 70, 97, 106, 118, 124, 127, 133, 172, 173, 178, 179, 205, 206, 214, 215, 19, 20, 25, 26, 10, 11, 16, 17]
[152, 42, 44, 65, 71, 98, 107, 119, 125, 128, 134, 171, 173, 177, 179, 204, 206, 213, 215, 18, 20, 24, 26, 9, 11, 15, 17]
[153, 45, 46, 72, 75, 81, 99, 108, 111, 126, 129, 180, 181, 183, 184, 189, 190, 207, 208, 18, 19, 21, 22, 0, 1, 3, 4]
[154, 46, 47, 73, 76, 82